<a href="https://colab.research.google.com/github/jimin0/2023_JUMP_AI/blob/main/%EC%8B%A0%EC%95%BD%EA%B0%9C%EB%B0%9C_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# If you have installed dgl-cuXX package, please uninstall it first.
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 MB 10.8 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=7b5664eb841e84ef3086

In [ ]:
!pip install deepchem dgl dgllife lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import deepchem as dc
import lightning as L

from torch.optim import AdamW
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import ModelCheckpoint
from deepchem.models.torch_models import MPNNModel
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# splitter의 종류를 고르고 고른 splitter를 이용해서 kfold로 데이터를 나누는 과정
def smiles_split(df, smiles, seed=42, k_fold=5, splitter='scaffold'):
  import deepchem as dc
  Xs, ys = np.arange(len(smiles)), np.ones(len(smiles))
  dataset = dc.data.DiskDataset.from_numpy(X=Xs, y=ys, w=np.zeros(len(smiles)), ids=smiles)
  if splitter == 'random':
    splitter = dc.splits.RandomSplitter()
  elif splitter == 'scaffold':
    splitter = dc.splits.Scaffoldsplitter()
  elif splitter == 'fingerprints':
    splitter = dc.splits.FingerprintSplitter()
  folds = splitter.k_fold_split(dataset, k=k_fold, seed=seed)
  dfs = []
  for fold in folds:
    train_indices = fold[0].X
    val_indices = fold[1].X
    train_df = df.iloc[train_indices].reset_index(drop=True)
    val_df = df.iloc[val_indices].reset_index(drop=True)
    dfs.append((train_df, val_df))
  return dfs

# deepchem의 splitter의 종류와 기능

In [ ]:
BATCH_SIZE = 32
SEED = 42
K_FOLD = 5

In [ ]:
L.seed_everything(SEED)

INFO: Global seed set to 42
INFO:lightning.fabric.utilities.seed:Global seed set to 42


42

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/2023_신약개발대회/open/test.csv', encoding='UTF-8')
test_df = pd.read_csv('/content/drive/MyDrive/2023_신약개발대회/open/test.csv', encoding='UTF-8')

In [ ]:
test_df.columns

Index(['id', 'SMILES', 'AlogP', 'Molecular_Weight', 'Num_H_Acceptors',
       'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea'],
      dtype='object')

In [ ]:
# target 값 column에 추가
test_df['MLM'] = 0
test_df['HLM'] = 0

In [ ]:
train_smiles = train_df['SMILES'].tolist()
# 화학식..
#train_smiles

###deepchem splitter의 종류

RandomSplitter: 데이터를 무작위로 분할합니다. 주어진 비율에 따라 훈련, 검증 및 테스트 세트를 생성할 수 있습니다.

ScaffoldSplitter: 데이터를 화학 구조의 기준으로 분할합니다. 이 방법은 화학적으로 유사한 화합물이 동일한 분할 세트에 속하도록 보장합니다.

ButinaSplitter: 화학적 유사성을 기반으로 데이터를 분할합니다. 이 방법은 데이터를 유사한 화합물 군집으로 그룹화하여 각 군집을 다른 분할 세트에 할당합니다.

TimeSplitter: 시간에 따라 데이터를 분할합니다. 이 방법은 시계열 데이터나 시간에 종속적인 데이터에 유용합니다.

StratifiedSplitter: 데이터를 클래스 또는 라벨에 따라 분할합니다. 클래스 불균형이 있는 분류 문제에 사용할 수 있습니다.

FingerprintSplitter: 화학적 지문(fingerprint)을 기반으로 데이터를 분할합니다. 이 방법은 화학 구조의 유사성을 고려하여 분할합니다.

In [ ]:
# train set과 validation set을 분리
for train_fold, val_fold in smiles_split(train_df, train_smiles, seed=SEED, k_fold=K_FOLD, splitter='fingerprints'):
    break

In [ ]:
# for문을 이용해서 각 column을 scaling시켜줌(target을 제외한 나머지)
for col in ['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea']:
  scaler = MinMaxScaler()
  train_fold[col] = scaler.fit_transform(train_fold[[col]].values).reshape(-1)
  val_fold[col] = scaler.transform(val_fold[[col]].values).reshape(-1)
  test_df[col] = scaler.transform(test_df[[col]].values).reshape(-1)

# nan값을 train set의 mean값으로 채워주기
  train_nan_value = train_fold[col].mean()
  train_fold[col] = train_fold[col].fillna(train_nan_value)
  val_fold[col] = val_fold[col].fillna(train_nan_value)
  test_df[col] = test_df[col].fillna(train_nan_value)

In [ ]:
# deepchem을 사용하기 위한 featurizer
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

In [ ]:
test_df.columns

Index(['id', 'SMILES', 'AlogP', 'Molecular_Weight', 'Num_H_Acceptors',
       'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea', 'MLM', 'HLM'],
      dtype='object')

In [ ]:
# 분자구조의 특징을 vector로 변환해서 출력하는 과정
train_X = featurizer.featurize(train_fold['SMILES'].tolist())
train_w = train_fold[['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea']].values
train_dataset = dc.data.NumpyDataset(X=train_X, y=train_fold[['MLM','HLM']].values, w=train_w) # 메모리에서 추출 후 numpy로 바꿔줌

val_X = featurizer.featurize(val_fold['SMILES'].tolist())
val_w = val_fold[['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea']].values
val_dataset = dc.data.NumpyDataset(X=val_X, y=val_fold[['MLM','HLM']].values, w=val_w)

test_X = featurizer.featurize(test_df['SMILES'].tolist())
test_w = test_df[['AlogP','Molecular_Weight','Num_H_Acceptors','Num_H_Donors','Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea']].values
test_dataset = dc.data.NumpyDataset(X=test_X, y=test_df[['MLM','HLM']].values, w=test_w)

In [ ]:
# tensor형태로 바꿔주는 함수
def collate_fn(samples):
    X = [sample[0] for sample in samples]
    y = torch.Tensor([sample[1] for sample in samples])
    w = torch.Tensor([sample[2] for sample in samples])
    return ([X],y,w)

In [ ]:
train_datas = []
val_datas = []
test_datas = []

for x,y,w in zip(train_dataset.X, train_dataset.y, train_dataset.w):
    train_datas.append((x,y,w))

for x,y,w in zip(val_dataset.X, val_dataset.y, val_dataset.w):
    val_datas.append((x,y,w))

for x,y,w in zip(test_dataset.X, test_dataset.y, test_dataset.w):
    test_datas.append((x,y,w))

train_dataloader = DataLoader(train_datas, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(val_datas, batch_size=BATCH_SIZE*2, collate_fn=collate_fn)
test_dataloader = DataLoader(test_datas, batch_size=BATCH_SIZE*2, collate_fn=collate_fn)

In [ ]:
class LitMPNNSMILESClassification(L.LightningModule):
    def __init__(self, batch_size, node_out_feats=64, n_tasks=2):
        super().__init__()
        model = MPNNModel(
            mode='regression',
            n_tasks=n_tasks,
            node_out_feats=node_out_feats,
            batch_size=batch_size,
        )
        self._prepare_batch = lambda batch : model._prepare_batch(batch)
        self.model = model.model
        self.model.model.predict = nn.Identity()
        self.batch_size=batch_size
        self.classifier = nn.Sequential(
            nn.LazyLinear(node_out_feats//2),
            nn.ReLU(),
            nn.LazyLinear(n_tasks)
        )

        self.validation_step_outputs = []

    def forward(self, x, xp):
        x = self.model(x)
        x = torch.cat([x, xp], dim=-1)
        x_out = self.classifier(x)
        return x_out

    def training_step(self, batch, batch_idx):
        x, *_ = self._prepare_batch(batch)
        y_true, xp = batch[1]/100, batch[2]
        y_pred = self(x, xp)
        loss1 = F.mse_loss(y_pred[:,0].flatten(), y_true[:,0].flatten())
        loss2 = F.mse_loss(y_pred[:,1].flatten(), y_true[:,1].flatten())
        loss = (loss1**0.5 + loss2**0.5)/2
        self.log_dict({"train_loss": loss}, on_step=True, prog_bar=True, batch_size=self.batch_size)
        return loss

    def validation_step(self, batch, batch_idx):
        x, *_ = self._prepare_batch(batch)
        y_true, xp = batch[1], batch[2]
        y_pred = self(x, xp)*100
        loss1 = F.mse_loss(y_pred[:,0].flatten(), y_true[:,0].flatten())
        loss2 = F.mse_loss(y_pred[:,1].flatten(), y_true[:,1].flatten())
        loss = (loss1, loss2)
        self.validation_step_outputs.append(loss)
        return loss

    def on_validation_epoch_end(self):
        loss = torch.Tensor(self.validation_step_outputs)
        loss1, loss2 = loss[:, 0], loss[:, 1]
        loss = ((loss1.mean())**0.5 + (loss2.mean())**0.5)/2
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.validation_step_outputs.clear()

    def predict_step(self, batch, batch_idx):
        x, *_ = self._prepare_batch(batch)
        y_true, xp = batch[1], batch[2]
        y_pred = self(x, xp)*100
        return y_pred

    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=1e-3, betas=(0.9, 0.999), weight_decay=0.01)
        return optimizer

In [ ]:
lit_model = LitMPNNSMILESClassification(
    batch_size=BATCH_SIZE
)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoint/',
    filename='MPNN-{epoch:02d}-{train_loss:.2f}-{val_loss:.2f}',
    save_top_k=1,
)

In [ ]:
trainer = L.Trainer(
    accelerator='gpu',
#     precision='bf16-mixed',
    max_epochs=100,
    callbacks=[checkpoint_callback],
)


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


### cuda로 데이터 올린 후 dataloader로 모델학습시키기

In [ ]:
trainer.fit(lit_model, train_dataloader, val_dataloader)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /content/checkpoint exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:415: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  warning_cache.warn(
INFO: 
  | Name       | Type       | Params
------------------------------------------
0 | model      | MPNN       | 673 K 
1 | classifier | Sequential | 0     
------------------------------------------
673 K     Trainable params
0         Non-trainable params
673 K     Total params
2.693     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.mo

Sanity Checking: 0it [00:00, ?it/s]

<ipython-input-17-8161e30f2cb8>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  y = torch.Tensor([sample[1] for sample in samples])


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
lit_model = LitMPNNSMILESClassification.load_from_checkpoint(
    '/content/checkpoint/MPNN-epoch=22-train_loss=0.27-val_loss=33.14.ckpt',
    batch_size=BATCH_SIZE
)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
preds = trainer.predict(lit_model, test_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/2023 신약개발 대회/open (1)/sample_submission.csv')

In [ ]:
submission[['MLM','HLM']] = torch.cat(preds).numpy()

In [ ]:
submission.to_csv('submission_ver1',index=False)